In [1]:
!pip install datasets trl accelerate transformers bitsandbytes torch sentencepiece


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [19]:
!pip install huggingface_hub
from huggingface_hub import notebook_login
notebook_login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
from transformers import BitsAndBytesConfig
import torch

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [4]:
!pip install protobuf


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [23]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3",add_eos_token=True)
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", quantization_config = bnb_config)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:786: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
from datasets import load_dataset

ds = load_dataset("gopalkalpande/bbc-news-summary")
ds

DatasetDict({
    train: Dataset({
        features: ['File_path', 'Articles', 'Summaries'],
        num_rows: 2224
    })
})

In [7]:
ds = ds.remove_columns('File_path')
ds

DatasetDict({
    train: Dataset({
        features: ['Articles', 'Summaries'],
        num_rows: 2224
    })
})

In [8]:
!pip install scikit-learn


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [9]:
from datasets import Dataset, load_dataset, DatasetDict
from sklearn.model_selection import train_test_split

df = ds['train'].to_pandas()
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)
train_ds = DatasetDict({
    'train': Dataset.from_pandas(train_df)
})

test_ds = DatasetDict({
    'test': Dataset.from_pandas(test_df)
})
print(f"Training dataset size: {len(train_ds['train'])}")
print(f"Test dataset size: {len(test_ds['test'])}")


Training dataset size: 1668
Test dataset size: 556


In [10]:
split_ds = DatasetDict({
    'train': train_ds,
    'test': test_ds
})

print(split_ds)

DatasetDict({
    train: DatasetDict({
        train: Dataset({
            features: ['Articles', 'Summaries', '__index_level_0__'],
            num_rows: 1668
        })
    })
    test: DatasetDict({
        test: Dataset({
            features: ['Articles', 'Summaries', '__index_level_0__'],
            num_rows: 556
        })
    })
})


In [11]:
def format_example(example):
    return {
        'text': f"<s>[INST] {example['Articles']} [/INST] <s>[INST] {example['Summaries']} [/INST]",
        'summary': example['Summaries']
    }

In [12]:
formatted_train_ds = train_ds.map(format_example)
formatted_test_ds = test_ds.map(format_example)

formatted_split_ds = DatasetDict({
    'train': formatted_train_ds,
    'test': formatted_test_ds
})

print(formatted_split_ds)


Map:   0%|          | 0/1668 [00:00<?, ? examples/s]

Map:   0%|          | 0/556 [00:00<?, ? examples/s]

DatasetDict({
    train: DatasetDict({
        train: Dataset({
            features: ['Articles', 'Summaries', '__index_level_0__', 'text', 'summary'],
            num_rows: 1668
        })
    })
    test: DatasetDict({
        test: Dataset({
            features: ['Articles', 'Summaries', '__index_level_0__', 'text', 'summary'],
            num_rows: 556
        })
    })
})


In [13]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32768, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((

In [14]:
!pip install peft


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [15]:
from peft import LoraConfig, TaskType, get_peft_model
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  # Include feedforward components
    task_type=TaskType.CAUSAL_LM,
)

model = get_peft_model(model, lora_config)

In [16]:
model.add_adapter(peft_config=lora_config,adapter_name="ADAPTER")


In [17]:
formatted_split_ds['train']

DatasetDict({
    train: Dataset({
        features: ['Articles', 'Summaries', '__index_level_0__', 'text', 'summary'],
        num_rows: 1668
    })
})

In [20]:
from trl import SFTTrainer
from transformers import TrainingArguments
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.padding_side='right'
torch.cuda.empty_cache()

max_seq_length = 1024
train_dataset = formatted_split_ds['train']['train']
test_dataset = formatted_split_ds['test']['test']
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    warmup_steps=1,
    max_steps=200,
    learning_rate=2e-5,
    fp16=True,
    logging_steps=4,
    push_to_hub=True,
    output_dir='./mistral-7b-v0.3-summarizer',
    optim="paged_adamw_8bit",
)

trainer = SFTTrainer(
    model=model,
    dataset_text_field="text",  
    eval_dataset = test_dataset,
    train_dataset=train_dataset,
    args=training_args,
    peft_config=lora_config,
    tokenizer = tokenizer
)

model.config.use_cache = False
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1668 [00:00<?, ? examples/s]

Map:   0%|          | 0/556 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:407: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
4,1.953600
8,1.922400
12,1.835400
16,1.776200
20,1.712300
24,1.651500
28,1.675100
32,1.628900
36,1.621800
40,1.579400


TrainOutput(global_step=200, training_loss=1.5255442976951599, metrics={'train_runtime': 2023.7572, 'train_samples_per_second': 3.162, 'train_steps_per_second': 0.099, 'total_flos': 2.3149815032286413e+17, 'train_loss': 1.5255442976951599, 'epoch': 3.8369304556354917})

In [21]:
trainer.model.save_pretrained("mistral-7b-v0.3-summarizer")
trainer.tokenizer.save_pretrained("mistral-7b-v0.3-summarizer")

('mistral-7b-v0.3-summarizer/tokenizer_config.json',
 'mistral-7b-v0.3-summarizer/special_tokens_map.json',
 'mistral-7b-v0.3-summarizer/tokenizer.model',
 'mistral-7b-v0.3-summarizer/added_tokens.json',
 'mistral-7b-v0.3-summarizer/tokenizer.json')

In [24]:
new_model = "mistral-7b-v0.3-summarizer"
base_model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.3", quantization_config = bnb_config)
merged_model= PeftModel.from_pretrained(model, new_model)
merged_model= merged_model.merge_and_unload()
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

merged_model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/devesh-2002/mistral-7b-v0.3-summarizer/commit/3eef4a1b6f85cd78d53830b5c9dabf33d6aeaabe', commit_message='Upload tokenizer', commit_description='', oid='3eef4a1b6f85cd78d53830b5c9dabf33d6aeaabe', pr_url=None, pr_revision=None, pr_num=None)